In [1]:
import json
import re
from konlpy.tag import Okt
from collections import Counter
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
from wordcloud import WordCloud
import pandas as pd

import glob
from functools import reduce
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from wordcloud import WordCloud, STOPWORDS

import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim 
import warnings
warnings.filterwarnings('ignore')

C:\ProgramData\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


# 데이터 정리
### 영화정보
> meta19 meta20 meta21

### 리뷰
> 네이버 : n19 | n20 | n21<br>

## 데이터 정보
* title : 영화 제목
* genre : 장르
* nation : 국가
* running_time : 러닝타임
* release_date : 개봉일
* director : 감독
* actor : 출연진
* grade : 관람가 등급
* summary : 줄거리
* poster : 포스터 url
* score : 평점
* content : 리뷰 내용
* write_date : 리뷰 작성일
* source_site : 플랫폼

# 데이터 병합

In [6]:
meta19 = pd.read_csv('movie2019_last_of_last.csv')

In [7]:
meta20 = pd.read_csv('movie2020_last_of_last.csv')

In [8]:
meta21 = pd.read_csv('movie2021_last_of_last.csv')

In [9]:
n19 = pd.read_csv('naver_review2019.csv')

In [4]:
n20 = pd.read_csv('naver_review2020_source.csv')

In [5]:
n21 = pd.read_csv('naver_review2021.csv')

In [10]:
meta19.head(1)

,title,genre,nation,running_time,release_date,director,actor,grade,summary,poster
0,0.0MHz,공포,한국,102분,2019 .05.29 개봉,유선동,"정은지(소희), 이성열(상엽), 최윤영(윤정)",15세 관람가,"에디슨의 마지막 발명품, 유령 탐지기?\n “증명할 수 없는 미스터리란 없다”\n ...",https://movie-phinf.pstatic.net/20190517_285/1...


In [11]:
meta20.head(1)

,title,genre,nation,running_time,release_date,director,actor,grade,summary,poster
0,108명의 여자와 섹스하는 방법,"코미디, 드라마",일본,101분,2020 .05.21 개봉,마츠오 스즈키,"마츠오 스즈키, 나카야마 미호",청소년 관람불가,유명 시나리오 작가인 '카이바 고로'는 SNS 통해 아내 '아야코'의 외도 사실을 ...,https://movie-phinf.pstatic.net/20200512_12/15...


In [12]:
meta21.head(1)

,title,genre,nation,running_time,release_date,director,actor,grade,summary,poster
0,007 노 타임 투 다이,액션,미국,163분,2021 .09.29 개봉,캐리 후쿠나가,"다니엘 크레이그(제임스 본드), 라미 말렉(사핀), 라샤나 린치(노미)",12세 관람가,가장 강력한 운명의 적과 마주하게된 제임스 본드의 마지막 미션이 시작된다,https://movie-phinf.pstatic.net/20211001_121/1...


In [20]:
meta192021 = meta192021.reset_index(drop=True)

In [26]:
meta192021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1918 entries, 0 to 1917
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         1918 non-null   object
 1   genre         1918 non-null   object
 2   nation        1918 non-null   object
 3   running_time  1918 non-null   object
 4   release_date  1918 non-null   object
 5   director      1918 non-null   object
 6   actor         1918 non-null   object
 7   grade         1918 non-null   object
 8   summary       1918 non-null   object
 9   poster        1918 non-null   object
dtypes: object(10)
memory usage: 150.0+ KB


In [21]:
n19.head(1)

,title,score,content,write_date,release_date,source_site
0,0.0MHz,1,와 정말 무서운집 수준의 공포영화가 한국에 다시 나왔네요. 꼭 보러 가세요.착석하신...,2019.05.29 13:30,2019 .05.29 개봉,naver


In [22]:
n1920 = pd.merge(n19, n20, on=('title','score','content','write_date','release_date','source_site'),how='outer')

In [23]:
n192021 = pd.merge(n1920, n21, on=('title','score','content','write_date','release_date','source_site'),how='outer')

In [24]:
n192021 = n192021.reset_index(drop=True)

In [25]:
n192021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278847 entries, 0 to 278846
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   title         278847 non-null  object
 1   score         278847 non-null  int64 
 2   content       268004 non-null  object
 3   write_date    278847 non-null  object
 4   release_date  278847 non-null  object
 5   source_site   278847 non-null  object
dtypes: int64(1), object(5)
memory usage: 12.8+ MB


In [2]:
movie_n = pd.read_csv('naver_movie_192021.csv')

# 전처리
* dropna (10,843개)
* 스포일러 리뷰 삭제 (9,599개)

In [27]:
n192021 = n192021.dropna()

In [30]:
n192021 = n192021[~n192021['content'].str.contains('스포일러가')]

#### csv 저장

In [31]:
n192021.to_csv('naver_review_192021.csv',index=False)

In [32]:
meta192021.to_csv('meta192021.csv',index=False)

In [34]:
movie_all = pd.merge(meta192021, n192021
    , how = 'inner'
    , on = ("title","release_date"))

In [48]:
movie_alldrop.to_csv('naver_movie_192021.csv',index=False)

In [112]:
movie_alldrop.groupby('title').count()

,genre,nation,running_time,release_date,director,actor,grade,summary,poster,score,content,write_date,source_site,pre_content
title,,,,,,,,,,,,,,
#살아있다,397,397,397,397,397,397,397,397,397,397,397,397,397,397
#아이엠히어,208,208,208,208,208,208,208,208,208,208,208,208,208,208
0.0MHz,490,490,490,490,490,490,490,490,490,490,490,490,490,490
007 노 타임 투 다이,437,437,437,437,437,437,437,437,437,437,437,437,437,437
1 더하기 3,39,39,39,39,39,39,39,39,39,39,39,39,39,39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
헬보이,489,489,489,489,489,489,489,489,489,489,489,489,489,489
협객,6,6,6,6,6,6,6,6,6,6,6,6,6,6
호라이즌 라인,52,52,52,52,52,52,52,52,52,52,52,52,52,52


---

## 중복값 제거
* 중복된 리뷰가 수집된 경우 drop

In [46]:
movie_alldrop = movie_all.drop_duplicates().reset_index(drop=True)

In [49]:
movie_alldrop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235785 entries, 0 to 235784
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   title         235785 non-null  object
 1   genre         235785 non-null  object
 2   nation        235785 non-null  object
 3   running_time  235785 non-null  object
 4   release_date  235785 non-null  object
 5   director      235785 non-null  object
 6   actor         235785 non-null  object
 7   grade         235785 non-null  object
 8   summary       235785 non-null  object
 9   poster        235785 non-null  object
 10  score         235785 non-null  int64 
 11  content       235785 non-null  object
 12  write_date    235785 non-null  object
 13  source_site   235785 non-null  object
dtypes: int64(1), object(13)
memory usage: 25.2+ MB


#### 리뷰 X 영화정보 merge : 
데이터 총 `234785개`


In [5]:
movie_n[movie_n['title']=='기생충'].head(1)

,title,genre,nation,running_time,release_date,director,actor,grade,summary,poster,score,content,write_date,source_site
14556,기생충,드라마,한국,131분,2019 .05.30 개봉,봉준호,"송강호(기택), 이선균(동익), 조여정(연교)",15세 관람가,전원백수로 살 길 막막하지만 사이는 좋은 기택(송강호) 가족.\n 장남 기우(최우식...,https://movie-phinf.pstatic.net/20190528_36/15...,10,"비에 젖지 않는 고급 장난감 텐트와, 비에 젖다 못해 잠겨버리는 반지하 가구",2019.05.30 15:58,naver


In [7]:
movie_n[movie_n['title']=='기생충'].tail(2)

,title,genre,nation,running_time,release_date,director,actor,grade,summary,poster,score,content,write_date,source_site
15036,기생충,드라마,한국,131분,2019 .05.30 개봉,봉준호,"송강호(기택), 이선균(동익), 조여정(연교)",15세 관람가,전원백수로 살 길 막막하지만 사이는 좋은 기택(송강호) 가족.\n 장남 기우(최우식...,https://movie-phinf.pstatic.net/20190528_36/15...,1,정말 재미없다. 이게 영화인가?,2019.06.04 11:51,naver
15037,기생충,드라마,한국,131분,2019 .05.30 개봉,봉준호,"송강호(기택), 이선균(동익), 조여정(연교)",15세 관람가,전원백수로 살 길 막막하지만 사이는 좋은 기택(송강호) 가족.\n 장남 기우(최우식...,https://movie-phinf.pstatic.net/20190528_36/15...,1,으.. 으.. 으.. 으.. 으.. 으.. 으.. 으.. 으.. 으.. 으.. 으....,2019.06.02 14:20,naver


In [50]:
movie_alldrop

,title,genre,nation,running_time,release_date,director,actor,grade,summary,poster,score,content,write_date,source_site
0,0.0MHz,공포,한국,102분,2019 .05.29 개봉,유선동,"정은지(소희), 이성열(상엽), 최윤영(윤정)",15세 관람가,"에디슨의 마지막 발명품, 유령 탐지기?\n “증명할 수 없는 미스터리란 없다”\n ...",https://movie-phinf.pstatic.net/20190517_285/1...,1,와 정말 무서운집 수준의 공포영화가 한국에 다시 나왔네요. 꼭 보러 가세요.착석하신...,2019.05.29 13:30,naver
1,0.0MHz,공포,한국,102분,2019 .05.29 개봉,유선동,"정은지(소희), 이성열(상엽), 최윤영(윤정)",15세 관람가,"에디슨의 마지막 발명품, 유령 탐지기?\n “증명할 수 없는 미스터리란 없다”\n ...",https://movie-phinf.pstatic.net/20190517_285/1...,10,"내용 구성이나 연출, cg는 저예산 영화 정도의 수준이었지만 정은지가 연기를 잘한다",2019.05.29 10:58,naver
2,0.0MHz,공포,한국,102분,2019 .05.29 개봉,유선동,"정은지(소희), 이성열(상엽), 최윤영(윤정)",15세 관람가,"에디슨의 마지막 발명품, 유령 탐지기?\n “증명할 수 없는 미스터리란 없다”\n ...",https://movie-phinf.pstatic.net/20190517_285/1...,2,상엽이 형은 어쩌다가 어릴때 죽었는지 다리 피흘리는 여고생 연관있을줄 알았는데 연관...,2019.05.29 20:15,naver
3,0.0MHz,공포,한국,102분,2019 .05.29 개봉,유선동,"정은지(소희), 이성열(상엽), 최윤영(윤정)",15세 관람가,"에디슨의 마지막 발명품, 유령 탐지기?\n “증명할 수 없는 미스터리란 없다”\n ...",https://movie-phinf.pstatic.net/20190517_285/1...,10,정은지 연기 소름...,2019.05.29 12:21,naver
4,0.0MHz,공포,한국,102분,2019 .05.29 개봉,유선동,"정은지(소희), 이성열(상엽), 최윤영(윤정)",15세 관람가,"에디슨의 마지막 발명품, 유령 탐지기?\n “증명할 수 없는 미스터리란 없다”\n ...",https://movie-phinf.pstatic.net/20190517_285/1...,6,일단 정은지는 완전 합격점. 최윤영도 좋았다. 여배우 둘 빼면 에휴...게다가 초반...,2019.05.29 11:34,naver
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235780,혼자 사는 사람들,드라마,한국,90분,2021 .05.19 개봉,홍성은,"공승연(진아), 정다은(수진)",12세 관람가,"집에서도 밖에서도 늘 혼자가 편한 진아.\n 사람들은 자꾸 말을 걸어오지만, 진아는...",https://movie-phinf.pstatic.net/20210506_173/1...,10,ㅅㅂ 돈도 ㅈ도 안주는데 ㅈ같ㅇ 영화다. 참는것도 한계가있지 갑질하는새끼들은 지출신...,2021.08.22 04:37,naver
235781,혼자 사는 사람들,드라마,한국,90분,2021 .05.19 개봉,홍성은,"공승연(진아), 정다은(수진)",12세 관람가,"집에서도 밖에서도 늘 혼자가 편한 진아.\n 사람들은 자꾸 말을 걸어오지만, 진아는...",https://movie-phinf.pstatic.net/20210506_173/1...,10,"바쁜 현대인들 삶과 , 삶에 대해 전하고자 하시는 내용이 너무 잘 표현된거 같다는 ...",2021.08.22 03:30,naver
235782,혼자 사는 사람들,드라마,한국,90분,2021 .05.19 개봉,홍성은,"공승연(진아), 정다은(수진)",12세 관람가,"집에서도 밖에서도 늘 혼자가 편한 진아.\n 사람들은 자꾸 말을 걸어오지만, 진아는...",https://movie-phinf.pstatic.net/20210506_173/1...,7,지독하게 현실감이 넘쳐서 영화 같지가 않았다.그리고 진아가 누굴까 하고 검색했는데 ...,2021.08.22 02:51,naver
235783,혼자 사는 사람들,드라마,한국,90분,2021 .05.19 개봉,홍성은,"공승연(진아), 정다은(수진)",12세 관람가,"집에서도 밖에서도 늘 혼자가 편한 진아.\n 사람들은 자꾸 말을 걸어오지만, 진아는...",https://movie-phinf.pstatic.net/20210506_173/1...,10,두렵고 무서운 영화에요,2021.08.22 02:41,naver


# 영화 장르 정리
movie['genre'].unique()
* 공포
* 멜로/로맨스
* 드라마
* 범죄
* 가족
* 코미디
* 액션
* 스릴러
* 애니메이션
* 모험
* 미스터리
* SF
* 전쟁
* 판타지
* 서스펜스
* 뮤지컬
* 블랙코미디

---

# 불용어 설정

* 한국불용어 100 + 영화, '진짜', '많이', '정말', '그냥', '그', '애', '생각', '완전', '편','내용','중간', '꼭','편','모두', ...제외
https://bab2min.tistory.com/544#google_vignette

In [61]:
len(stop_words)

124

In [60]:
stop_words = list(set(stop_words))

In [8]:
stop_words= ['또','너무','넘','이게','계속','영화','분','꼭', '하', '받', '않', '말', '알','진짜', '많이', '정말', '그냥', '그', '애', '생각', '완전', '편','내용',
             '중간', '꼭','편','상미', '모두' , '거','더', '가', '뭐', '그렇', '말하', '중', '때', '되', '암', '보', '있', '개', '볼','일',
             '안','진짜', '듯', '위하', '대하', '년', '쫌', '위', '없', '주', '같', '나', '정말','ㅠㅠ','ㅎㅎ','근데','하고','저는'
             '지', '걸', '못','좀', '그', '왜','때문', '임', '게', '것', '의', '사람', '이','대한','아주','특히','ㅋㅋㅋ','이거',
             '수','이','있','하','것','들','그','되','수','보','않','나','사람','주','아니',
             '등','같','우리','때','년','가','한','지','대하','오','말','일','그렇','위하','때문',
             '그것','두','말하','알','그러나','그런데','그래도','그래서','받','못하','일',
             '그런','또','문제','더','사회','많','좋','크','중','나오','가지','씨','시키',
             '만들','지금','생각하','그러','속','하나','집','살','모르','적','월','데','자신',
             '안','어떤','내','경우','명','생각','시간','그녀','다시','이런','앞','보이','번',
             '나','다른','어떻','여자','남자','개','전','들','사실','이렇','점','싶','말','정도',
             '좀','원','잘','통하','소리','놓','영화','진짜', '많이', '정말', '그냥', '그', '애',
             '생각', '완전', '편','내용','중간', '꼭','편','상미', '모두' , '거']

stop_words = list(set(stop_words))

---

# 장르 통합 리뷰

In [91]:
allcontent = []
for words in tqdm(movie_alldrop['content']):
    # word_tokenize을 통해서 Tokenizer를 진행해줍니다. 
    word_tokens = word_tokenize(words)
    # re.sub('[^a-z0-9]+', ' ', str(word_tokens)) 명령어를 통해서 특수문자를 모두 제거해줍니다. 
    sentence = re.sub('[^A-Za-z0-9]+', ' ', str(word_tokens))
    # 양끝에 존재하는 공백을 제거해줍니다. 
    sentence = sentence.strip()

    # 문장의 Tokenizer를 진행해줍니다. 
    sentence_tokens = word_tokenize(sentence)
    result = ''
    for token in sentence_tokens: 
        if token not in stop_words:
            result += ' ' + token 
    result = result.strip().lower()
    allcontent.append(result)

  0%|          | 0/235785 [00:00<?, ?it/s]

In [92]:
movie_alldrop['pre_content'] = allcontent

In [93]:
from collections import namedtuple

# Doc2Vec 입력형태를 맞춰줍니다. 
# TaggedDocument 형태로 ('영화 내용', '제목') 을 받습니다. 
## 이후, TaggedDocument에 모든 문서에 대해서 '영화 내용', '제목'을 넣어줍니다. 
agg = movie_alldrop[['title', 'pre_content']]
TaggedDocument = namedtuple('TaggedDocument', 'words tags')
tagged_train_docs = [TaggedDocument((c), [d]) for d, c in agg[['title', 'pre_content']].values]

In [95]:
doc_vectorizer = doc2vec.Doc2Vec(
    dm=1,            # PV-DBOW / default 1
    #dbow_words=1,    # w2v simultaneous with DBOW d2v / default 0
    window=10,        # distance between the predicted word and context words
    vector_size=300,        # vector size
    alpha=0.025,     # learning-rate
    seed=1234,
    #min_count=5,    # ignore with freq lower
    min_alpha=0.025, # min learning-rate
    #workers=4,   # multi cpu
    #hs = 1,          # hierar chical softmax / default 0
    negative = 5   # negative sampling / default 5
)

In [96]:
# 위에서 만든 tagged_train_docs으로 사전을 만들어줍니다. 
doc_vectorizer.build_vocab(tagged_train_docs)
print(str(doc_vectorizer))

Doc2Vec(dm/m,d300,n5,w10,mc5,s0.001,t3)


In [97]:
# To save
doc_vectorizer.save('doc2vec_all.model')

In [98]:
# 벡터 문서 학습
from time import time

start = time()

for epoch in tqdm(range(30)):
    # Doc2Vec 학습을 진행하는데 Learning rate를 계속 감소해주면서 학습을 진행해줍니다. 
    doc_vectorizer.train(tagged_train_docs, total_examples=doc_vectorizer.corpus_count, epochs=30)
    doc_vectorizer.alpha -= 0.002 # decrease the learning rate
    doc_vectorizer.min_alpha = doc_vectorizer.alpha # fix the learning rate, no decay

end = time()
print("During Time: {}".format(end-start))

  0%|          | 0/30 [00:00<?, ?it/s]

During Time: 8826.567391395569


In [102]:
doc_vectorizer.docvecs.most_similar('기생충', topn=20)

[('프린세스 에미: 마법 책의 비밀', 0.6510905027389526),
 ('대환장파티', 0.6344231367111206),
 ('맨슨 패밀리', 0.6096524000167847),
 ('내 완벽한 남사친의 비밀', 0.6074991822242737),
 ('프리즌 브레이커스', 0.5989813804626465),
 ('영화로운 나날', 0.5903171896934509),
 ('보르도 우정여행', 0.5771888494491577),
 ('파이널리스트', 0.5678276419639587),
 ('프론테라스', 0.562434732913971),
 ('7월7일', 0.5503718256950378),
 ('체크 히어로', 0.5486385822296143),
 ('슈퍼문', 0.5434455275535583),
 ('더 배니싱', 0.5404161214828491),
 ('고골: 악령과의 전쟁', 0.5395044684410095),
 ('O.J. 심슨 사건 파일', 0.5385341048240662),
 ('구르는 수레바퀴', 0.5361682772636414),
 ('초대녀 2', 0.5360038876533508),
 ('그리고 우린 춤을 추었다', 0.535868227481842),
 ('기묘한 이야기: 리플레이', 0.5341373682022095),
 ('얼굴들', 0.5340752601623535)]

In [106]:
doc_vectorizer.docvecs.most_similar('보스 베이비 2', topn=20)

[('그것: 두 번째 이야기', 0.576075553894043),
 ('트롤: 월드 투어', 0.5725052952766418),
 ('스웨그', 0.5673669576644897),
 ('초미의 관심사', 0.5517657995223999),
 ('컨저링 3: 악마가 시켰다', 0.5465413331985474),
 ('닥터 두리틀', 0.5164274573326111),
 ('범털2 : 쩐의 전쟁', 0.513810396194458),
 ('겨울왕국 2', 0.509134829044342),
 ('콰이어트 플레이스 2', 0.501354992389679),
 ('데드풀2: 순한 맛', 0.4988044798374176),
 ('오시로가의 비밀', 0.49445825815200806),
 ('날씨의 아이', 0.49288317561149597),
 ('극장판 짱구는 못말려: 격돌! 낙서왕국과 얼추 네 명의 용사들', 0.4908711314201355),
 ('걸후드', 0.4896414875984192),
 ('나의 흑역사 로맨티카', 0.48686811327934265),
 ('메모리즈', 0.4853954613208771),
 ('스네이크 아이즈: 지.아이.조', 0.48079225420951843),
 ('피닉스', 0.47633570432662964),
 ('완벽한 그녀의 비밀', 0.4735615849494934),
 ('신의 한 수: 귀수편', 0.47197017073631287)]

---

# 장르 : 공포

## → 장르가 공포를 포함하는 모든 영화
## 공포'만' 해당X

In [64]:
horror = movie_alldrop[movie_alldrop['genre'].str.contains('공포')].reset_index(drop=True)

In [ ]:
from gensim.models import doc2vec
# 사전 구축
doc_vectorizer = doc2vec.Doc2Vec(size=300, alpha=0.025, min_alpha=0.025, seed=1234)
doc_vectorizer.build_vocab(tagged_train_docs)

In [11]:
from gensim.models import doc2vec

In [12]:
from nltk.corpus import stopwords 
from tqdm.notebook import tqdm
from nltk.tokenize import word_tokenize, sent_tokenize

---

# 공포영화만 유사도 측정하기

In [65]:
hcontent = []
for words in tqdm(horror['content']):
    # word_tokenize을 통해서 Tokenizer를 진행해줍니다. 
    word_tokens = word_tokenize(words)
    # re.sub('[^a-z0-9]+', ' ', str(word_tokens)) 명령어를 통해서 특수문자를 모두 제거해줍니다. 
    sentence = re.sub('[^A-Za-z0-9]+', ' ', str(word_tokens))
    # 양끝에 존재하는 공백을 제거해줍니다. 
    sentence = sentence.strip()

    # 문장의 Tokenizer를 진행해줍니다. 
    sentence_tokens = word_tokenize(sentence)
    result = ''
    for token in sentence_tokens: 
        if token not in stop_words:
            result += ' ' + token 
    result = result.strip().lower()
    hcontent.append(result)

  0%|          | 0/22291 [00:00<?, ?it/s]

In [ ]:
from gensim.models import doc2vec
# 사전 구축
doc_vectorizer = doc2vec.Doc2Vec(size=300, alpha=0.025, min_alpha=0.025, seed=1234)
doc_vectorizer.build_vocab(tagged_train_docs)

In [66]:
horror['pre_content'] = hcontent

In [ ]:
## 장르 X
## 줄거리 + 리뷰 

In [13]:
doc_vectorizer = doc2vec.Doc2Vec(
    dm=1,            # PV-DBOW / default 1
    #dbow_words=1,    # w2v simultaneous with DBOW d2v / default 0
    window=10,        # distance between the predicted word and context words
    vector_size=300,        # vector size
    alpha=0.025,     # learning-rate
    seed=1234,
    #min_count=5,    # ignore with freq lower
    min_alpha=0.025, # min learning-rate
    #workers=4,   # multi cpu
    #hs = 1,          # hierar chical softmax / default 0
    negative = 5   # negative sampling / default 5
)

In [68]:
from collections import namedtuple

# Doc2Vec 입력형태를 맞춰줍니다. 
# TaggedDocument 형태로 ('영화 내용', '제목') 을 받습니다. 
## 이후, TaggedDocument에 모든 문서에 대해서 '영화 내용', '제목'을 넣어줍니다. 
agg = horror[['title', 'pre_content']]
TaggedDocument = namedtuple('TaggedDocument', 'words tags')
tagged_train_docs = [TaggedDocument((c), [d]) for d, c in agg[['title', 'pre_content']].values]

In [69]:
# 위에서 만든 tagged_train_docs으로 사전을 만들어줍니다. 
doc_vectorizer.build_vocab(tagged_train_docs)
print(str(doc_vectorizer))

Doc2Vec(dm/m,d300,n5,w10,mc5,s0.001,t3)


In [79]:
# To save
doc_vectorizer.save('doc2vec.model')

In [75]:
# 벡터 문서 학습
from time import time

start = time()

for epoch in tqdm(range(30)):
    # Doc2Vec 학습을 진행하는데 Learning rate를 계속 감소해주면서 학습을 진행해줍니다. 
    doc_vectorizer.train(tagged_train_docs, total_examples=doc_vectorizer.corpus_count, epochs=30)
    doc_vectorizer.alpha -= 0.002 # decrease the learning rate
    doc_vectorizer.min_alpha = doc_vectorizer.alpha # fix the learning rate, no decay

end = time()
print("During Time: {}".format(end-start))

  0%|          | 0/30 [00:00<?, ?it/s]

During Time: 605.9310328960419


In [76]:
horror['title'].unique()

array(['0.0MHz', '47미터 2', '공포의 묘지', '귀신의 향기', '그레이브 하우스', '나이트메어 시네마',
       '노크: 낯선 그림자', '늑대의 아이들', '다커 나이트', '닥터 킬러 패밀리', '더 바이 바이 맨',
       '더 보이', '더 빌리지', '더 스쿨', '더하우스', '데드 돈 다이', '디 엔드?',
       '레드 스프링 아포칼립스', '로드킬', '룩 어웨이', '매리', '매트리악', '맨디', '미드소마',
       '미이라 리턴즈', '베스와 베라', '변신', '보어 살인 멧돼지', '사일런스', '사자', '사탄의 인형',
       '사탄의 인형: 상속 받은 저주', '살인마 잭의 집', '악령의 캠핑카', '암전',
       '애나벨 : 잊혀지지 않는 선율', '애나벨 집으로', '어스', '언데드 인 커밍', '요로나의 저주',
       '위치 오브 하우스', '이스케이프 룸', '인면어: 저주의 시작', '좀비 : 죽은 자들의 도시',
       '좀비랜드: 더블 탭', '카르마', '카리스마타 살인', '카메라를 멈추면 안 돼! 스핀오프 : 할리우드 대작전',
       '카운트다운', '캐빈 오브 더 컨저링', '컨벤트', '크로우 라이징', '파라노말 액티비티 2019',
       '파이와켓: 죽음의 주문', '폐교', '폴라로이드', '프로디지', '해피 데스데이 2 유', '헌팅 오브 힐하우스',
       '검은 사제들: 지옥의 문', '겟 아웃 얼라이브', '고스트 오브 워', '그곳', '그집', '기기괴괴 성형수',
       '기묘한 이야기: 리플레이', '나이트메어 라디오', '나이트메어 서커스', '나치 오버로드', '뉴 뮤턴트',
       '더 렌탈: 소리없는 감시자', '더 보이 2: 돌아온 브람스', '더 터닝', '데빌 인사이드 미', '돈 스피크',
       '디어스킨', '라비드', '런 런 런', '레벨 16', '맨슨

In [83]:
doc_vectorizer.docvecs.most_similar(positive=['컨저링: 귀신의 집', '공포의 묘지'], negative=['47미터 2'], topn=20)

[('인면어: 저주의 시작', 0.5635553002357483), ('세인트 아가타', 0.5156008005142212), ('자매의 공동묘지', 0.4939689338207245), ('매트리악', 0.4887956976890564), ('맨디', 0.48686933517456055), ('링: 더 라스트 챕터', 0.46589788794517517), ('데빌 인사이드 미', 0.46528691053390503), ('더 스쿨', 0.46294263005256653), ('레드 스프링 아포칼립스', 0.46280795335769653), ('헌티드 파크', 0.46124446392059326), ('돈 스피크', 0.45582059025764465), ('존 도우', 0.4442197382450104), ('목격자 : 눈이 없는 아이', 0.4434628486633301), ('더 나이트', 0.44225889444351196), ('악플러 죽이기', 0.44203585386276245), ('실종: 사라진 사람들', 0.42329758405685425), ('요가학원:죽음의쿤달리니', 0.42286810278892517), ('악령의 캠핑카', 0.41867345571517944), ('런 런 런', 0.41380488872528076), ('피어썸', 0.40190389752388)]

In [78]:
doc_vectorizer.docvecs.most_similar('컨저링 3: 악마가 시켰다', topn=20)

[('아이 씨 유', 0.650468647480011),
 ('실종: 사라진 사람들', 0.6181316375732422),
 ('카르마', 0.6146055459976196),
 ('그곳', 0.6142624020576477),
 ('애나벨 : 잊혀지지 않는 선율', 0.6089791059494019),
 ('컬트 걸', 0.6086001396179199),
 ('캐빈 오브 더 컨저링', 0.6045429110527039),
 ('신 강시선생', 0.6001917719841003),
 ('해피 데스데이 2 유', 0.5811768770217896),
 ('피어 오브 레인', 0.5806698203086853),
 ('더 배니싱', 0.5653866529464722),
 ('47미터 2', 0.5523381233215332),
 ('위자 크래프트', 0.5474677681922913),
 ('파라노말 액티비티 2019', 0.5408865213394165),
 ('살인 매뉴얼', 0.5378244519233704),
 ('크로우 라이징', 0.5057649612426758),
 ('리틀 조', 0.5034528374671936),
 ('더 빌리지', 0.4987199902534485),
 ('닥터 킬러 패밀리', 0.4970897138118744),
 ('고스트 오브 워', 0.4949326515197754)]

In [89]:
doc_vectorizer.docvecs.most_similar('파라노말 액티비티 2019', topn=20)

[('블러드: 궁금한 이야기', 0.9891312122344971),
 ('캐빈 오브 더 컨저링', 0.8981232643127441),
 ('리틀 조', 0.863559365272522),
 ('기묘한 이야기: 리플레이', 0.8523256182670593),
 ('카르마', 0.8510911464691162),
 ('애나벨 : 잊혀지지 않는 선율', 0.8502852916717529),
 ('맨슨 패밀리', 0.8493539094924927),
 ('그곳', 0.8033229112625122),
 ('아이 씨 유', 0.7965874671936035),
 ('신 강시선생', 0.7965527176856995),
 ('더 배니싱', 0.7948595285415649),
 ('더 빌리지', 0.7891238927841187),
 ('피어 오브 레인', 0.7856281995773315),
 ('위자 크래프트', 0.7809276580810547),
 ('살인 매뉴얼', 0.7757599353790283),
 ('크로우 라이징', 0.7675133943557739),
 ('나이트메어 서커스', 0.7662829160690308),
 ('사라져버린', 0.7643951773643494),
 ('닥터 킬러 패밀리', 0.7599900364875793),
 ('컬트 걸', 0.757792055606842)]

---

# 장르별 유사도 영화 추천 상위 10개 

In [424]:
documentary = movie_n[movie_n['genre'].str.contains('다큐멘터리')]

In [425]:
rcontent = []
for words in tqdm(documentary['content']):
    # word_tokenize을 통해서 Tokenizer를 진행해줍니다. 
    word_tokens = word_tokenize(words)
    # re.sub('[^a-z0-9]+', ' ', str(word_tokens)) 명령어를 통해서 특수문자를 모두 제거해줍니다. 
    sentence = re.sub('[^A-Za-z0-9]+', ' ', str(word_tokens))
    # 양끝에 존재하는 공백을 제거해줍니다. 
    sentence = sentence.strip()

    # 문장의 Tokenizer를 진행해줍니다. 
    sentence_tokens = word_tokenize(sentence)
    result = ''
    for token in sentence_tokens: 
        if token not in stop_words:
            result += ' ' + token 
    result = result.strip().lower()
    rcontent.append(result)

  0%|          | 0/11904 [00:00<?, ?it/s]

In [426]:
documentary['pre_content'] = rcontent

In [427]:
doc_vectorizer = doc2vec.Doc2Vec(
    dm=1,            # PV-DBOW / default 1
    #dbow_words=1,    # w2v simultaneous with DBOW d2v / default 0
    window=10,        # distance between the predicted word and context words
    vector_size=300,        # vector size
    alpha=0.025,     # learning-rate
    seed=1234,
    #min_count=5,    # ignore with freq lower
    min_alpha=0.025, # min learning-rate
    #workers=4,   # multi cpu
    #hs = 1,          # hierar chical softmax / default 0
    negative = 5   # negative sampling / default 5
)

In [428]:
from collections import namedtuple

# Doc2Vec 입력형태를 맞춰줍니다. 
# TaggedDocument 형태로 ('영화 내용', '제목') 을 받습니다. 
## 이후, TaggedDocument에 모든 문서에 대해서 '영화 내용', '제목'을 넣어줍니다. 
agg = documentary[['title', 'pre_content']]
TaggedDocument = namedtuple('TaggedDocument', 'words tags')
tagged_train_docs = [TaggedDocument((c), [d]) for d, c in agg[['title', 'pre_content']].values]

In [429]:
# 위에서 만든 tagged_train_docs으로 사전을 만들어줍니다. 
doc_vectorizer.build_vocab(tagged_train_docs)
print(str(doc_vectorizer))

Doc2Vec(dm/m,d300,n5,w10,mc5,s0.001,t3)


In [430]:
# 벡터 문서 학습
from time import time

start = time()

for epoch in tqdm(range(30)):
    # Doc2Vec 학습을 진행하는데 Learning rate를 계속 감소해주면서 학습을 진행해줍니다. 
    doc_vectorizer.train(tagged_train_docs, total_examples=doc_vectorizer.corpus_count, epochs=30)
    doc_vectorizer.alpha -= 0.002 # decrease the learning rate
    doc_vectorizer.min_alpha = doc_vectorizer.alpha # fix the learning rate, no decay

end = time()
print("During Time: {}".format(end-start))

  0%|          | 0/30 [00:00<?, ?it/s]

During Time: 227.02488660812378


---

In [44]:
aaa = doc_vectorizer.docvecs.most_similar('엠마', topn=20)

In [46]:
aaa[0][0]

'키싱 부스 2'

In [222]:
slist = family.title.unique().tolist()

In [223]:
slist[:5]

['13년의 공백', 'EBS 댄스파티!', '겨울왕국 2', '고릴라와 슈퍼레이스', '구스 베이비']

---

### genre_list 
* title_x : 유사도를 예측하기 위해 입력하는 영화 제목
* title_y : 입력한 영화 제목의 유사도가 높은 영화 제목
* similarity : 입력과 출력 영화의 유사도

In [431]:
result = []
def genre_list(genre):
    # 전체 데이터에서 입력한 장르만 포함된 데이터 프레임 추출
    mg = movie_n[movie_n['genre'].str.contains(genre)]
    # 제목 리스트 생성
    mgli = mg.title.unique().tolist()
    #print(mgli) 
    for i in range(0,len(mgli)): # 영화의 수 만큼 
        mgsim = []
        # 제목을 입력했을 때 10개의 추천영화가 도출
        mmm = doc_vectorizer.docvecs.most_similar(mgli[i], topn=10) 
        # 데이터 프레임 생성
        df = pd.DataFrame(mmm, columns =['title_y', 'similarity'])
        df['title_x'] = mgli[i]
        df['genre'] = genre
        result.append(df)

In [432]:
genre_list('다큐멘터리')

---

### 공포

In [157]:
horror = pd.concat(result)

In [198]:
horror.iloc[65:70,:]

,title_y,similarity,title_x,genre
5,맨디,0.717732,노크: 낯선 그림자,공포
6,레드 스프링 아포칼립스,0.693068,노크: 낯선 그림자,공포
7,컨저링: 귀신의 집,0.658900,노크: 낯선 그림자,공포
8,더 배니싱,0.652452,노크: 낯선 그림자,공포
9,얼티밋 배틀,0.650097,노크: 낯선 그림자,공포


In [299]:
horror[horror['similarity']> 0.8]

,title_y,similarity,title_x,genre
0,존 도우,0.806622,노크: 낯선 그림자,공포
1,데스 숏컷,0.806510,노크: 낯선 그림자,공포
0,크로우 라이징,0.824017,더 바이 바이 맨,공포
1,할로윈: 살인마 잭,0.822003,더 바이 바이 맨,공포
0,크로우 라이징,0.971301,더 빌리지,공포
...,...,...,...,...
6,더 빌리지,0.918931,피어 오브 레인,공포
7,크로우 라이징,0.876482,피어 오브 레인,공포
8,카르마,0.859899,피어 오브 레인,공포
9,할로윈: 살인마 잭,0.851856,피어 오브 레인,공포


In [300]:
horror = horror.reset_index(drop=True)
horror.to_csv('horror.csv',index=False)

### 로맨스

In [171]:
romance = pd.concat(result)

In [193]:
romance.iloc[47:52,:]

,title_y,similarity,title_x,genre
7,이별의 목적,0.757375,계절과 계절 사이,멜로/로맨스
8,큐리오사,0.741941,계절과 계절 사이,멜로/로맨스
9,클라라,0.740128,계절과 계절 사이,멜로/로맨스
0,네가 내가 되었으면 좋겠다,0.742626,굿바이 썸머,멜로/로맨스
1,내 사랑 찾기,0.620386,굿바이 썸머,멜로/로맨스


In [301]:
romance = romance.reset_index(drop=True)
romance.to_csv('romance.csv',index=False)

### 드라마

In [184]:
drama = pd.concat(result)

In [192]:
drama.iloc[98:103,:]

,title_y,similarity,title_x,genre
8,엔젤 오브 마인,0.788466,계절과 계절 사이,드라마
9,슈팅걸스,0.776903,계절과 계절 사이,드라마
0,1917,0.594529,"고흐, 영원의 문에서",드라마
1,더블패티,0.573229,"고흐, 영원의 문에서",드라마
2,아이 엠 우먼,0.568890,"고흐, 영원의 문에서",드라마


In [302]:
drama = drama.reset_index(drop=True)
drama.to_csv('drama.csv',index=False)

### 범죄

In [208]:
crime = pd.concat(result)

In [212]:
crime.iloc[24:29,:]

,title_y,similarity,title_x,genre
4,킬러의 보디가드 2,0.636436,나쁜 녀석들: 더 무비,범죄
5,디스트로이어,0.631680,나쁜 녀석들: 더 무비,범죄
6,에이바,0.605774,나쁜 녀석들: 더 무비,범죄
7,21 브릿지: 테러 셧다운,0.604939,나쁜 녀석들: 더 무비,범죄
8,비행,0.594132,나쁜 녀석들: 더 무비,범죄


In [303]:
crime = crime.reset_index(drop=True)
crime.to_csv('crime.csv',index=False)

### 가족

In [224]:
family = pd.concat(result)

In [225]:
family.iloc[98:103,:]

,title_y,similarity,title_x,genre
8,베카신!,0.779718,동키 킹,가족
9,구스 베이비,0.764487,동키 킹,가족
0,원더랜드,0.847956,라이온 킹,가족
1,닥터 두리틀,0.806366,라이온 킹,가족
2,썬키스 패밀리,0.785979,라이온 킹,가족


In [304]:
family = family.reset_index(drop=True)
family.to_csv('family.csv',index=False)

### 코미디

In [235]:
comedy = pd.concat(result)

In [246]:
comedy.iloc[133:138,:]

,title_y,similarity,title_x,genre
3,잃어버린 세계를 찾아서,0.615852,극한직업,코미디
4,크루엘라,0.609393,극한직업,코미디
5,기묘한 가족,0.605210,극한직업,코미디
6,슈퍼문,0.600795,극한직업,코미디
7,내 사랑 찾기,0.582706,극한직업,코미디


In [305]:
comedy = comedy.reset_index(drop=True)
comedy.to_csv('comedy.csv',index=False)

### 액션

In [256]:
action = pd.concat(result)

In [261]:
action.iloc[40:45,:]

,title_y,similarity,title_x,genre
0,카르텔 2045,0.781806,고검기담: 소명신검의 부활,액션
1,척살대,0.775853,고검기담: 소명신검의 부활,액션
2,몬스터 가디언즈,0.770202,고검기담: 소명신검의 부활,액션
3,나치 오버로드,0.749156,고검기담: 소명신검의 부활,액션
4,잘리카투,0.694453,고검기담: 소명신검의 부활,액션


In [306]:
action = action.reset_index(drop=True)
action.to_csv('action.csv',index=False)

### 스릴러

In [271]:
thriller= pd.concat(result)

In [272]:
thriller.iloc[98:103,:]

,title_y,similarity,title_x,genre
8,목격자들 5번의 살인,0.056162,나노 테러리즘: CIA 비밀요원,스릴러
9,라플라스의 마녀,0.056131,나노 테러리즘: CIA 비밀요원,스릴러
0,에브리타임 아이 다이,0.611840,나를 찾아줘,스릴러
1,사냥의 시간,0.607749,나를 찾아줘,스릴러
2,팬데믹,0.571659,나를 찾아줘,스릴러


In [307]:
thriller = thriller.reset_index(drop=True)
thriller.to_csv('thriller.csv',index=False)

### 애니메이션

In [283]:
animation= pd.concat(result)

In [287]:
animation.iloc[12:17,:]

,title_y,similarity,title_x,genre
2,신밧드와 마법 양탄자,0.904160,고릴라와 슈퍼레이스,애니메이션
3,모험왕 블링키,0.890328,고릴라와 슈퍼레이스,애니메이션
4,헬로 몽키,0.879091,고릴라와 슈퍼레이스,애니메이션
5,동키 킹,0.865685,고릴라와 슈퍼레이스,애니메이션
6,똑딱똑딱 마법의 시계나라,0.858593,고릴라와 슈퍼레이스,애니메이션


In [308]:
animation = animation.reset_index(drop=True)
animation.to_csv('animation.csv',index=False)

### 모험

In [297]:
adventure= pd.concat(result)

In [298]:
adventure.iloc[98:103,:]

,title_y,similarity,title_x,genre
8,툼 인베이더,0.584473,레드슈즈,모험
9,극장판 도라에몽: 진구의 신공룡,0.576830,레드슈즈,모험
0,기묘한 이야기: 리플레이,0.882317,리노,모험
1,체크 히어로,0.877668,리노,모험
2,플레이모빌: 더 무비,0.793840,리노,모험


In [309]:
adventure = adventure.reset_index(drop=True)
adventure.to_csv('adventure.csv',index=False)

---

### 미스터리

In [319]:
mystery= pd.concat(result)

In [323]:
mystery.iloc[25:30,:]

,title_y,similarity,title_x,genre
25,프로페서 앤 매드맨,0.672261,노벰버 크리미널즈,미스터리
26,블러드하운드,0.671214,노벰버 크리미널즈,미스터리
27,비블리아 고서당 사건수첩,0.669880,노벰버 크리미널즈,미스터리
28,오시로가의 비밀,0.662793,노벰버 크리미널즈,미스터리
29,파리의 딜릴리,0.649068,노벰버 크리미널즈,미스터리


In [321]:
mystery = mystery.reset_index(drop=True)
mystery.to_csv('mystery.csv',index=False)

### SF

In [333]:
sf= pd.concat(result)

In [334]:
sf.iloc[98:103,:]

,title_y,similarity,title_x,genre
8,포제서,0.730940,몬스터 가디언즈,SF
9,심장소리,0.709116,몬스터 가디언즈,SF
0,더 수어사이드 스쿼드,0.525683,샤잠!,SF
1,캡틴 마블,0.510713,샤잠!,SF
2,마션 인베이션,0.508875,샤잠!,SF


In [335]:
sf = sf.reset_index(drop=True)
sf.to_csv('sf.csv',index=False)

### 전쟁

In [345]:
war= pd.concat(result)

In [346]:
war.iloc[98:103,:]

,title_y,similarity,title_x,genre
8,패트리어트 워,0.749042,솔져스 메모리,전쟁
9,1917,0.746085,솔져스 메모리,전쟁
0,바이킹 워,0.941131,스페셜 솔져,전쟁
1,터키쉬 아이스크림,0.930223,스페셜 솔져,전쟁
2,패트리어트 워,0.922119,스페셜 솔져,전쟁


In [347]:
war = war.reset_index(drop=True)
war.to_csv('war.csv',index=False)

### 판타지

In [357]:
fantasy= pd.concat(result)

In [366]:
fantasy.iloc[80:85,:]

,title_y,similarity,title_x,genre
80,드래곤 빌리지,0.612016,덤보,판타지
81,사자,0.550013,덤보,판타지
82,적인걸6 - 집요술사,0.545669,덤보,판타지
83,킹 아더 카멜롯의 기사,0.542180,덤보,판타지
84,영화로운 나날,0.539652,덤보,판타지


In [360]:
fantasy = fantasy.reset_index(drop=True)
fantasy.to_csv('fantasy.csv',index=False)

### 서스펜스

In [376]:
suspense= pd.concat(result)

In [387]:
suspense.iloc[7:12,:]

,title_y,similarity,title_x,genre
1,신문기자,-0.104747,신영웅문 : 천년의 비밀,서스펜스
2,좋은 사람,-0.105865,신영웅문 : 천년의 비밀,서스펜스
0,신문기자,0.988590,좋은 사람,서스펜스
1,명탐정 코난:전율의 악보,0.879516,좋은 사람,서스펜스
2,신영웅문 : 천년의 비밀,-0.105865,좋은 사람,서스펜스


In [390]:
suspense = suspense.reset_index(drop=True)
suspense.to_csv('suspense.csv',index=False)

### 뮤지컬

In [400]:
musical= pd.concat(result)

In [406]:
musical.iloc[70:75,:]

,title_y,similarity,title_x,genre
0,잃어버린 얼굴 1895,0.924100,아이스,뮤지컬
1,당나귀 공주,0.858261,아이스,뮤지컬
2,슈퍼스타 뚜루,0.753403,아이스,뮤지컬
3,롤라,0.708973,아이스,뮤지컬
4,블루스 인 인디아,0.686905,아이스,뮤지컬


In [ ]:
musical = musical.reset_index(drop=True)
musical.to_csv('musical.csv',index=False)

### 블랙코미디

In [416]:
blackcomedy= pd.concat(result)

In [419]:
blackcomedy

,title_y,similarity,title_x,genre


In [423]:
movie_n[movie_n['genre'].str.contains('블랙코미디')].groupby('title')['title'].count()

title
폴리스 스파이    70
Name: title, dtype: int64

In [417]:
blackcomedy.iloc[98:103,:]

,title_y,similarity,title_x,genre


In [ ]:
blackcomedy = blackcomedy.reset_index(drop=True)
blackcomedy.to_csv('blackcomedy.csv',index=False)

### 다큐멘터리

In [433]:
documentary= pd.concat(result)

In [434]:
documentary.iloc[70:75,:]

,title_y,similarity,title_x,genre
0,스쿨 오브 락(樂),0.995592,녹차의 중력,다큐멘터리
1,발리: 천상의 울림,0.993866,녹차의 중력,다큐멘터리
2,머리카락,0.985007,녹차의 중력,다큐멘터리
3,스매시 앤 그랩: 보석강도단 핑크 팬더,0.978416,녹차의 중력,다큐멘터리
4,백두 번째 구름,0.958807,녹차의 중력,다큐멘터리


In [435]:
documentary = documentary.reset_index(drop=True)
documentary.to_csv('documentary.csv',index=False)

In [467]:
genresim = genresim.append(documentary, ignore_index=True)

In [468]:
genresim

,title_y,similarity,title_x,genre
0,위자 악마소환게임,0.658763,0.0MHz,공포
1,이블데드 2020,0.654926,0.0MHz,공포
2,메가 블리자드,0.616077,0.0MHz,공포
3,나치 오버로드,0.565864,0.0MHz,공포
4,컨벤트,0.563100,0.0MHz,공포
...,...,...,...,...
27937,쿠엔틴 타란티노 8,0.576438,학교 가는 길,다큐멘터리
27938,스매시 앤 그랩: 보석강도단 핑크 팬더,0.574950,학교 가는 길,다큐멘터리
27939,스쿨 오브 락(樂),0.568384,학교 가는 길,다큐멘터리
27940,녹차의 중력,0.564385,학교 가는 길,다큐멘터리


In [469]:
genresim.to_csv('genresim.csv',index=False)